# import necessery libraries

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import KFold

# Dataset loading

In [2]:
# Define the dataset path
dataset_path = 'dog_vs_cat_dataset/animals' 

# Set up image data generator
data_gen = ImageDataGenerator(rescale=1./255)

# Load all data into a single generator (shuffle to mix the images)
data_generator = data_gen.flow_from_directory(
    dataset_path,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    shuffle=True  # Shuffle the data
)

# Load all images and labels into memory
X, y = [], []
for i in range(len(data_generator)):
    batch_data, batch_labels = data_generator[i]
    X.extend(batch_data)
    y.extend(batch_labels)
X = np.array(X)
y = np.array(y)

Found 1000 images belonging to 2 classes.


# Define the CNN model

In [3]:
def create_model(learning_rate):
    model = keras.Sequential([
        keras.layers.InputLayer(input_shape=(128, 128, 3)),
        
        keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Dropout(0.2), 
        
        keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Dropout(0.2), 

        keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Dropout(0.2), 

        keras.layers.Flatten(),
        keras.layers.Dense(512, activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(1, activation='sigmoid')
        
       
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy', 'Precision', 'Recall']
    )
    return model

# Perform K-Fold Cross-Validation
k = 5  # Number of folds
kf = KFold(n_splits=k, shuffle=True, random_state=42)

fold = 1
for train_index, val_index in kf.split(X):
    print(f"Training on Fold {fold}...")

    # Split the data into training and validation sets
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Create the model
    model = create_model(learning_rate=0.001)

    # Set up early stopping
    early_stopping = EarlyStopping(patience=5, restore_best_weights=True)

    # Train the model
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=30,
        batch_size=32,
        callbacks=[early_stopping]
    )

    # Evaluate the model on the validation set
    val_loss, val_accuracy, val_precision, val_recall = model.evaluate(X_val, y_val)
    print(f"Fold {fold} - Validation Loss: {val_loss}, Accuracy: {val_accuracy}, Precision: {val_precision}, Recall: {val_recall}")
    
    fold += 1

Training on Fold 1...


C:\Users\asmaa\anaconda3\Lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Epoch 1/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 16s 412ms/step - Precision: 0.7076 - Recall: 0.6873 - accuracy: 0.6920 - loss: 0.7956 - val_Precision: 0.4600 - val_Recall: 1.0000 - val_accuracy: 0.4600 - val_loss: 1.6579
Epoch 2/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 361ms/step - Precision: 0.8015 - Recall: 0.7892 - accuracy: 0.7938 - loss: 0.4408 - val_Precision: 0.5412 - val_Recall: 1.0000 - val_accuracy: 0.6100 - val_loss: 0.6143
Epoch 3/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 358ms/step - Precision: 0.8650 - Recall: 0.8641 - accuracy: 0.8634 - loss: 0.3161 - val_Precision: 0.9348 - val_Recall: 0.4674 - val_accuracy: 0.7400 - val_loss: 0.4768
Epoch 4/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 379ms/step - Precision: 0.8992 - Recall: 0.9050 - accuracy: 0.9024 - loss: 0.2107 - val_Precision: 1.0000 - val_Recall: 0.4565 - val_accuracy: 0.7500 - val_loss: 0.3938
Epoch 5/30
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 367ms/step - Precision: 0.9842 - Recall: 0.9227 - accuracy: 0.9510 - loss: 0.1376 - val_Precision: 1.0000 - val_Recall